In [214]:
import psycopg2

In [218]:
#fetch isbn numbers from the database, then get their histories
con = psycopg2.connect("dbname='nytimes'") 
cur = con.cursor()    
cur.execute("SELECT title,description FROM books")
rows = cur.fetchall()
con.close()

In [220]:
description_list = dict()
#print type(description_list)
for row in rows:
    description_list[row[0]]= row[1]  

In [221]:
titles = description_list.keys()
descriptions = description_list.values()

In [222]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
import re

In [223]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [224]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in description_list.values():
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [225]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

SyntaxError: invalid syntax (<ipython-input-225-d87b491533da>, line 2)

In [199]:
vocab_frame.head()

,words
a,a
death,death
at,at
an,an
elementari,elementary


In [202]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(description_list.values()) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 357 ms, sys: 33.5 ms, total: 390 ms
Wall time: 365 ms
(523, 130)


In [213]:
terms = tfidf_vectorizer.get_feature_names()
terms[:20]

[u"'s",
 u'account',
 u'actress',
 u'adapt',
 u'adventur',
 u'age',
 u'agent',
 u'america',
 u'american',
 u'anim',
 u'art',
 u'author',
 u'becom',
 u'biographi',
 u'book',
 u'boy',
 u'british',
 u'brother',
 u'career',
 u'case']

In [204]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [205]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 177 ms, sys: 3.1 ms, total: 180 ms
Wall time: 188 ms


In [1]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
#    print("Cluster %d titles:" % i, end='')
#    for title in vocab_frame.ix[i]['title'].values.tolist():
#        print(' %s,' % title, end='')
#    print() #add whitespace
#    print() #add whitespace
    
print()
print()

Top terms per cluster:



NameError: name 'km' is not defined